In [1]:
# ! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# ! wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
# ! tar xf spark-3.5.4-bin-hadoop3.tgz
# ! pip install -q findspark

In [2]:
import os
from itertools import combinations

import findspark
os.environ["SPARK_HOME"] = "spark-3.5.4-bin-hadoop3"
findspark.init()

import pyspark as spark
from pyspark import SparkContext

from pyspark.sql import (
    SparkSession,
    DataFrame,
    functions as f
)
from pyspark.sql.functions import (
    collect_list,
    explode,
    col,
    size,
)
from pyspark.sql.types import (
    ArrayType,
    IntegerType,
    StringType,
)

In [3]:
spark = SparkSession \
        .builder \
        .appName("task02c") \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("ERROR") 

your 131072x1 screen size is bogus. expect trouble
25/10/16 09:05:24 WARN Utils: Your hostname, DESKTOP-8IE5HJF resolves to a loopback address: 127.0.1.1; using 192.168.173.21 instead (on interface eth0)
25/10/16 09:05:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 09:05:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
file_path = "data/baskets.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df = df.dropna()

In [18]:
basket_df = df.groupBy("Member_number", "Date") \
              .agg(collect_list("itemDescription"))

item_col_name = "item_list"

basket_df = basket_df.toDF("id", "date", item_col_name) \
                     .select(item_col_name)

basket_df.show(10)

+--------------------+
|           item_list|
+--------------------+
|[sausage, whole m...|
|[whole milk, past...|
|[canned beer, mis...|
|[sausage, hygiene...|
|[soda, pickled ve...|
|[sausage, whole m...|
| [frankfurter, curd]|
|  [whole milk, soda]|
| [beef, white bread]|
|[frankfurter, sod...|
+--------------------+
only showing top 10 rows



In [19]:
from pcy import PCY, PCYModel

pcy_analyzer = PCY(min_support=100, min_confidence=0.1, num_buckets=1000, item_col=item_col_name)

pcy_model = pcy_analyzer.fit(basket_df)

print("\n--- Results from PCYModel ---")
pcy_model.show_frequent_pairs(n=10)
pcy_model.show_association_rules(n=10)


--- Results from PCYModel ---
Top 10 frequent pairs found:


+----------------+----------+-------+
|item1           |item2     |support|
+----------------+----------+-------+
|other vegetables|whole milk|243    |
|rolls/buns      |whole milk|227    |
|soda            |whole milk|199    |
|whole milk      |yogurt    |183    |
|other vegetables|rolls/buns|182    |
|other vegetables|soda      |160    |
|whole milk      |whole milk|148    |
|sausage         |whole milk|147    |
|tropical fruit  |whole milk|136    |
|other vegetables|yogurt    |131    |
+----------------+----------+-------+
only showing top 10 rows

Top 10 association rules found (ordered by confidence):
+--------------+----------+-------+-------------------+
|item1         |item2     |support|conf               |
+--------------+----------+-------+-------------------+
|bottled beer  |whole milk|113    |0.16448326055312956|
|sausage       |whole milk|147    |0.1590909090909091 |
|pip fruit     |whole milk|110    |0.1478494623655914 |
|shopping bags |whole milk|107    |0.1463748290013